# Preparación del dataset con STRING y Ensembl

En este notebook prepararemos el dataset con las interacciones físicas directas registradas en el dataset STRING filtrado por la especie homo sapiens y se emparejará cada ID de proteína a su secuencia de aminoacidos correspondiente para en el siguiente notebook hacer el preprocesamiento necesario antes de entrenar el modelo.

In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [2]:
!wget https://stringdb-static.org/download/protein.physical.links.full.v12.0/9606.protein.physical.links.full.v12.0.txt.gz
!gunzip 9606.protein.physical.links.full.v12.0.txt.gz
!mv 9606.protein.physical.links.full.v12.0.txt ../datasets/

--2025-06-08 19:05:06--  https://stringdb-static.org/download/protein.physical.links.full.v12.0/9606.protein.physical.links.full.v12.0.txt.gz
Resolviendo stringdb-static.org (stringdb-static.org)... 49.12.123.75
Conectando con stringdb-static.org (stringdb-static.org)[49.12.123.75]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 11766797 (11M) [application/octet-stream]
Guardando como: ‘9606.protein.physical.links.full.v12.0.txt.gz’

9606.protein.physic 100%[===================>]  11,22M  16,8MB/s    en 0,7s    

2025-06-08 19:05:07 (16,8 MB/s) - ‘9606.protein.physical.links.full.v12.0.txt.gz’ guardado [11766797/11766797]



In [3]:
!wget https://ftp.ensembl.org/pub/release-114/fasta/homo_sapiens/pep/Homo_sapiens.GRCh38.pep.all.fa.gz
!gunzip Homo_sapiens.GRCh38.pep.all.fa.gz
!mv Homo_sapiens.GRCh38.pep.all.fa ../datasets/

--2025-06-08 19:05:08--  https://ftp.ensembl.org/pub/release-114/fasta/homo_sapiens/pep/Homo_sapiens.GRCh38.pep.all.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 14966285 (14M) [application/x-gzip]
Guardando como: ‘Homo_sapiens.GRCh38.pep.all.fa.gz’

Homo_sapiens.GRCh38 100%[===================>]  14,27M  18,1MB/s    en 0,8s    

2025-06-08 19:05:09 (18,1 MB/s) - ‘Homo_sapiens.GRCh38.pep.all.fa.gz’ guardado [14966285/14966285]



## 1. Exploración de datos

In [4]:
import pandas as pd

db_string_path = '../datasets/9606.protein.physical.links.full.v12.0.txt'
df_string = pd.read_csv(db_string_path, sep=' ', low_memory=False)

df_string.head()

,protein1,protein2,homology,experiments,experiments_transferred,database,database_transferred,textmining,textmining_transferred,combined_score
0,9606.ENSP00000000233,9606.ENSP00000257770,0,312,0,0,0,0,0,311
1,9606.ENSP00000000233,9606.ENSP00000226004,0,162,0,0,0,0,0,161
2,9606.ENSP00000000233,9606.ENSP00000434442,0,0,0,500,0,0,0,499
3,9606.ENSP00000000233,9606.ENSP00000262455,0,531,0,0,0,0,0,531
4,9606.ENSP00000000233,9606.ENSP00000303145,0,0,0,500,0,0,0,499


In [5]:
df_string.shape

(1477610, 10)

In [6]:
df_string.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477610 entries, 0 to 1477609
Data columns (total 10 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   protein1                 1477610 non-null  object
 1   protein2                 1477610 non-null  object
 2   homology                 1477610 non-null  int64 
 3   experiments              1477610 non-null  int64 
 4   experiments_transferred  1477610 non-null  int64 
 5   database                 1477610 non-null  int64 
 6   database_transferred     1477610 non-null  int64 
 7   textmining               1477610 non-null  int64 
 8   textmining_transferred   1477610 non-null  int64 
 9   combined_score           1477610 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 112.7+ MB


In [7]:
df_string.isna().sum().sum()

np.int64(0)

## 2. Limpieza y preparación de datos
El dataset de STRING que hemos descargado indica en la propia web: "incl. distinction: direct vs. interologs". Esto significa que se indica qué interacciones son directas o por interología. Para nuestro proyecto buscamos aquellas directas comprobadas en humanos mediante técnicas experimentales. Estas son las que tienen experimentos registrados.

In [8]:
df_direct = df_string[df_string['experiments'] > 0]
print(f"Total de interacciones con evidencia experimental directa: {len(df_direct)}")
df_direct.head()

Total de interacciones con evidencia experimental directa: 947158


,protein1,protein2,homology,experiments,experiments_transferred,database,database_transferred,textmining,textmining_transferred,combined_score
0,9606.ENSP00000000233,9606.ENSP00000257770,0,312,0,0,0,0,0,311
1,9606.ENSP00000000233,9606.ENSP00000226004,0,162,0,0,0,0,0,161
3,9606.ENSP00000000233,9606.ENSP00000262455,0,531,0,0,0,0,0,531
5,9606.ENSP00000000233,9606.ENSP00000263265,0,292,0,0,0,0,0,292
6,9606.ENSP00000000233,9606.ENSP00000365686,0,221,0,0,0,0,0,221


Ahora que tenemos aquellas proteínas con interacción física directa, obtenemos el listado de proteínas únicas y nos quedamos con los pares de proteínas conocidos (estos serán nuestros casos positivos) para luego generar pares que no estén registrados (estos serán los casos negativos).

In [9]:
# Obtenemos todas las proteínas únicas
all_proteins = pd.unique(df_string[['protein1', 'protein2']].values.ravel())
print(f"Número de proteínas únicas: {len(all_proteins)}")

Número de proteínas únicas: 18767


In [10]:
positive_pairs = set(
    tuple(sorted([row['protein1'], row['protein2']]))
    for _, row in df_direct.iterrows()
)

In [11]:
import random

negative_pairs = set()

while len(negative_pairs) < len(df_direct):
    p1, p2 = random.sample(list(all_proteins), 2)
    pair = tuple(sorted([p1, p2]))
    if pair not in positive_pairs:
        negative_pairs.add(pair)

Acabamos de generar el set de casos negativos para nuestro modelo. Sin embargo, hay que aclarar que aquí nos encontramos ante uno de los mayores desafíos conceptuales de las PPI: los casos negativos no implica que esas dos proteínas se haya demostrado que no interactúen entre ellas, implica que su interacción no está recogida dentro del listado de interacciones conocidas. Esto nos dice que un par del set de negativos podría en realidad dar lugar a una interacción positiva si se hiciese una prueba experimental. Entonces, **asumimos que los pares "negativos" no es que no interactúen entre ellas, sino que desconocemos su interacción**.

In [12]:
df_negative = pd.DataFrame(list(negative_pairs), columns=['protein1', 'protein2'])
df_negative['label'] = 0

df_positive = df_direct[['protein1', 'protein2']].copy()
df_positive['label'] = 1

df_string_balanced = pd.concat([df_positive, df_negative], ignore_index=True)

Ahora traducimos cada proteína de su ID a su aminoácido correspondiente gracias al fichero FASTA de Ensembl.

In [13]:
from fasta_parser import FastaParser

fasta_parser = FastaParser("../datasets/Homo_sapiens.GRCh38.pep.all.fa")
protein_seqs = fasta_parser.to_dict()

print(f"Total de proteínas indexadas: {len(protein_seqs)}")

Total de proteínas indexadas: 123887


In [14]:
for i, (ensembl_id, seq) in enumerate(protein_seqs.items()):
    print(f"{ensembl_id}: {seq[:50]}...")
    if i >= 4:
        break

ENSP00000451468: XSQPHTKPSVFVMKNGTNVACLVKEFYPKDIRINLVSSKKITEFDPAIVI...
ENSP00000480116: XIQNPDPAVYQLRDSKSSDKSVCLFTDFDSQTNVSQSKDSDVYITDKTVL...
ENSP00000487742: XDLNKVFPPEVAVFEPSEAEISHTQKATLVCLATGFFPDHVELSWWVNGK...
ENSP00000488819: XDLNKVFPPEVAVFEPSEAEISHTQKATLVCLATGFFPDHVELSWWVNGK...
ENSP00000478873: XDLKNVFPPEVAVFEPSEAEISHTQKATLVCLATGFYPDHVELSWWVNGK...


In [15]:
df_string_balanced["sequence1"] = df_string_balanced["protein1"].apply(
    lambda p: protein_seqs.get(fasta_parser.extract_ensembl_id(p), None)
)
df_string_balanced["sequence2"] = df_string_balanced["protein2"].apply(
    lambda p: protein_seqs.get(fasta_parser.extract_ensembl_id(p), None)
)

In [16]:
df_string_balanced.shape

(1894316, 5)

In [17]:
df_string_balanced.isna().sum().sum()

np.int64(66485)

Observamos que hay 66626 NaNs en nuestro DF tras añadir las secuencias de aminoácidos. Investigamos qué ocurre.

In [18]:
from utils import analyze_missing_proteins

analyze_missing_proteins(df_string_balanced, protein_seqs)

🔍 Pares con al menos un ID faltante: 65869
🔍 IDs únicos no encontrados: 366

🔍 Ejemplos de IDs no encontrados:
['ENSP00000392700', 'ENSP00000414922', 'ENSP00000484940', 'ENSP00000362551', 'ENSP00000370208', 'ENSP00000498888', 'ENSP00000385519', 'ENSP00000471239', 'ENSP00000479069', 'ENSP00000482552']


Parece que el problema es que 366 proteínas del dataset de STRING no se encuentran en el archivo FASTA de Ensembl. Esto puede deberse a distintos factores, como que las proteínas hayan sido renombradas, retiradas o aún no incluídas en el fichero de Ensembl. Teniendo en cuenta que tenemos un dataset con 1.894.316 entradas, eliminar 65.869 no se considera un problema.

In [19]:
df_string_balanced.dropna(inplace=True)

In [20]:
import gc

del df_string
del df_direct
del df_negative
del df_positive
gc.collect()

# Guardamos el DataFrame balanceado
df_string_balanced.to_parquet("../datasets/df_string_balanced.parquet")

## Conclusión

En este primer notebook hemos creado nuestro dataset a partir de los datos de STRING y Ensembl, sin valores nulos y haciendo que los datos estén balanceado al crear como negativos pares de secuencias cuya interacción no es conocida.